In [ ]:
from flask import Flask, render_template, request, jsonify, session, redirect, url_for
import hashlib
import json
import time
import smtplib
import os
from email.message import EmailMessage
from collections import Counter
import socket

app = Flask(__name__)
app.secret_key = 'securekey'  

ADMIN_CREDENTIALS = {'admin': 'password123'} 

class Blockchain:
    def __init__(self):
        self.chain = []
        self.votes = []
        self.voted_ids = set()
        self.create_block(proof=1, previous_hash='0')

    def create_block(self, proof, previous_hash):
        block = {
            'index': len(self.chain) + 1,
            'timestamp': time.time(),
            'votes': self.votes,
            'proof': proof,
            'previous_hash': previous_hash
        }
        self.votes = []
        self.chain.append(block)
        return block

    def get_previous_block(self):
        return self.chain[-1]

    def proof_of_work(self, previous_proof):
        new_proof = 1
        check_proof = False
        while not check_proof:
            hash_operation = hashlib.sha256(str(new_proof**2 - previous_proof**2).encode()).hexdigest()
            if hash_operation[:4] == '0000':
                check_proof = True
            else:
                new_proof += 1
        return new_proof

    def hash(self, block):
        encoded_block = json.dumps(block, sort_keys=True).encode()
        return hashlib.sha256(encoded_block).hexdigest()

    def is_chain_valid(self, chain):
        previous_block = chain[0]
        block_index = 1
        while block_index < len(chain):
            block = chain[block_index]
            if block['previous_hash'] != self.hash(previous_block):
                return False
            previous_proof = previous_block['proof']
            proof = block['proof']
            hash_operation = hashlib.sha256(str(proof**2 - previous_proof**2).encode()).hexdigest()
            if hash_operation[:4] != '0000':
                return False
            previous_block = block
            block_index += 1
        return True

    def add_vote(self, voter_id, candidate, email):
        if voter_id in self.voted_ids:
            return False
        self.votes.append({'voter_id': voter_id, 'candidate': candidate})
        self.voted_ids.add(voter_id)
        send_confirmation_email(email, candidate)
        return True

    def get_winner(self):
        all_votes = [vote['candidate'] for block in self.chain for vote in block['votes']]
        if not all_votes:
            return "No votes yet"
        vote_count = Counter(all_votes)
        winner = max(vote_count, key=vote_count.get)
        return winner
    
    import os

OFFLINE_VOTES_FILE = "offline_votes.json"

def save_offline_vote(voter_id, candidate, email):
    """Stores offline votes in a JSON file while preventing duplicates"""
    offline_votes = load_offline_votes()

    # Prevent duplicate offline votes
    for vote in offline_votes:
        if vote["voter_id"] == voter_id:
            print(f"Voter {voter_id} already voted offline. Skipping duplicate entry.")
            return  

    offline_votes.append({"voter_id": voter_id, "candidate": candidate, "email": email})
    
    with open(OFFLINE_VOTES_FILE, "w") as file:
        json.dump(offline_votes, file)


import os
import json

OFFLINE_VOTES_FILE = "offline_votes.json"

def load_offline_votes():
    if os.path.exists(OFFLINE_VOTES_FILE):
        try:
            with open(OFFLINE_VOTES_FILE, 'r') as file:
                data = json.load(file)  
            
            if not isinstance(data, list):
                print("Error: Offline votes file format incorrect")
                return False  

            synced_votes = []
            for vote in data:
                if not isinstance(vote, dict):  
                    continue
                
                voter_id = vote.get('voter_id')
                candidate = vote.get('candidate')
                email = vote.get('email')

                if voter_id in blockchain.voted_ids:
                    continue  

                blockchain.add_vote(voter_id, candidate, email)
                synced_votes.append(vote)

            
         
            with open(OFFLINE_VOTES_FILE, "w") as file:
                     json.dump([], file)  

            return synced_votes


        except json.JSONDecodeError:
            print("Error: Invalid JSON format in offline votes file")
            return []  

        except PermissionError as e:
            print(f"PermissionError: {e}")
            return []  
    return []




def sync_offline_votes():
    """Syncs stored offline votes to the blockchain, ensuring no duplicates."""
    offline_votes = load_offline_votes()
    successful_sync = []

    for vote in offline_votes:
        if vote["voter_id"] not in blockchain.voted_ids:  
            blockchain.add_vote(vote["voter_id"], vote["candidate"], vote["email"])
            previous_block = blockchain.get_previous_block()
            proof = blockchain.proof_of_work(previous_block["proof"])
            previous_hash = blockchain.hash(previous_block)
            blockchain.create_block(proof, previous_hash)
            successful_sync.append(vote)

    
    if successful_sync:
        with open(OFFLINE_VOTES_FILE, "w") as file:
            json.dump([vote for vote in offline_votes if vote not in successful_sync], file)



import smtplib
from email.message import EmailMessage

def send_confirmation_email(receiver_email, candidate):
    sender_email = "elgortkaif@gmail.com"  
    app_password = "abse dosr mwwu dtrf"  

    msg = EmailMessage()
    msg["Subject"] = "Vote Confirmation ✅"
    msg["From"] = sender_email
    msg["To"] = receiver_email
    msg.set_content(f"Hello,\n\nThank you for voting! 🗳️\nYou have successfully voted for {candidate}.\n\nBest regards,\nElection Team")

    try:
        with smtplib.SMTP("smtp.gmail.com", 587) as server:
            server.starttls()  
            server.login(sender_email, app_password)  
            server.send_message(msg)
        print(f"✅ Email successfully sent to {receiver_email}!")
    except Exception as e:
        print(f"❌ Failed to send email: {e}")



def is_online():
    """Checks if the system has internet access."""
    try:
        
        socket.create_connection(("8.8.8.8", 53), timeout=2)
        return True
    except OSError:
        return False



blockchain = Blockchain()

@app.route('/')
def index():
    sync_offline_votes() 
    return render_template('index.html')


@app.route('/vote', methods=['POST'])
def vote():
    voter_id = request.form['voter_id']
    candidate = request.form['candidate']
    email = request.form['email']

    if is_online():
        if blockchain.add_vote(voter_id, candidate, email):
            previous_block = blockchain.get_previous_block()
            proof = blockchain.proof_of_work(previous_block["proof"])
            previous_hash = blockchain.hash(previous_block)
            blockchain.create_block(proof, previous_hash)
            return jsonify({"message": "Vote recorded successfully online."})
        else:
            return jsonify({"message": "Voter ID has already voted!"}), 400
    else:
        
        save_offline_vote(voter_id, candidate, email)
        return jsonify({"message": "Offline vote saved. Will sync when online."})




@app.route('/get_chain', methods=['GET'])
def get_chain():
    return jsonify({'chain': blockchain.chain, 'length': len(blockchain.chain)})

@app.route('/admin', methods=['GET', 'POST'])
def admin():
    if request.method == 'POST':
        username = request.form['username']
        password = request.form['password']
        if username in ADMIN_CREDENTIALS and ADMIN_CREDENTIALS[username] == password:
            session['admin'] = username
            return redirect(url_for('winner'))
        return 'Invalid credentials', 401
    return render_template('admin.html')



@app.route('/winner', methods=['GET'])
def winner():
    if 'admin' not in session:
        return redirect(url_for('admin'))
    
    all_votes = [vote['candidate'] for block in blockchain.chain for vote in block['votes']]
    vote_count = Counter(all_votes)
    
    return render_template('winner.html', winner=blockchain.get_winner(), vote_count=dict(vote_count))


@app.route('/sync_votes', methods=['GET'])
def sync_votes():
    if load_offline_votes():
        return jsonify({'message': 'Offline votes synchronized successfully!'})
    else:
        return jsonify({'message': 'No offline votes to sync. You are online.'})


@app.route('/logout')
def logout():
    session.pop('admin', None)  
    return redirect(url_for('index'))  

if __name__ == '__main__':
    app.run(debug=False, use_reloader=False)
 


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [17/Apr/2025 16:47:56] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Apr/2025 16:48:19] "POST /vote HTTP/1.1" 200 -


✅ Email successfully sent to avyay6868@gmail.com!
